# Demonstration: Reading and Plotting NXazint Files with plaid.nexus

This notebook demonstrates how to read and plot NXazint HDF5 files using the `plaid.nexus` module.

In [ ]:
# Import required libraries
import os
import h5py as h5
import matplotlib.pyplot as plt
import numpy as np
from plaid.nexus import get_nx_entry, get_nx_default, get_nx_signal, get_nx_axes, get_nx_sample

## Define functions for reading the data

Function for reading the 1d data

In [ ]:
def get_azint1d_data(fname):
    """Get azimuthally integrated 1D data from a NeXus file."""
    with h5.File(fname, "r") as f:
        azint1d = get_nx_entry(f,definition='NXazint1d')
        if azint1d is None:
            return None, None, (None, None)

        # get the data group
        data_gr = get_nx_default(azint1d)
        
        # get the axes group
        axes_gr = get_nx_axes(data_gr)
        x = axes_gr[-1][:]
        x_label = axes_gr[-1].attrs.get('long_name','radial axis')
        x_unit = axes_gr[-1].attrs.get('units','')
        if x_unit:
            x_label += f" ({x_unit})"
            
        # get the signal group
        signal_gr = get_nx_signal(data_gr)
        I = signal_gr[:]
        I_label = signal_gr.attrs.get('long_name','intensity')
        I_unit = signal_gr.attrs.get('units','')
        if I_unit:
            I_label += f" ({I_unit})"
        return x, I, (x_label, I_label)

Function for reading the 2D data (azimuthally binned)

In [ ]:
def get_azint2d_data(fname):
    """Get azimuthally integrated 2D data from a NeXus file."""
    with h5.File(fname, "r") as f:
        azint2d = get_nx_entry(f,definition='NXazint2d')
        if azint2d is None:
            return None, None, None, (None, None, None)

        # get the data group
        data_gr = get_nx_default(azint2d)
        
        # get the axes group
        axes_gr = get_nx_axes(data_gr)
        #  radial axis 
        x = axes_gr[-1][:]
        x_label = axes_gr[-1].attrs.get('long_name','radial axis')
        x_unit = axes_gr[-1].attrs.get('units','')
        if x_unit:
            x_label += f" ({x_unit})"
        # azimuthal axis
        azi = axes_gr[1][:]
        azi_label = axes_gr[1].attrs.get('long_name','azimuthal axis')
        azi_unit = axes_gr[1].attrs.get('units','')
        if azi_unit:
            azi_label += f" ({azi_unit})"
            
        # get the signal group
        signal_gr = get_nx_signal(data_gr)
        I = signal_gr[:]
        I_label = signal_gr.attrs.get('long_name','intensity')
        I_unit = signal_gr.attrs.get('units','')
        if I_unit:
            I_label += f" ({I_unit})"
        return azi, x, I, (azi_label, x_label, I_label)

Function for reading the sample data (sample name, time, temperature, etc)

In [ ]:
def get_sample_data(fname):
    """Get sample data from a NeXus file."""
    with h5.File(fname, "r") as f:
        entry = get_nx_entry(f)
        sample = get_nx_sample(entry)
        if sample is None:
            return None, None
        name = sample.get('name', '')
        if name:
            name = name[()]
            if isinstance(name, bytes):
                name = name.decode('utf-8')
        sample_data = {}
        for key in sample.keys():
            if isinstance(sample[key], h5.Dataset) and sample[key].ndim > 0:
                units = sample[key].attrs.get('units', '')
                if units:
                    label = f"{key} ({units})"
                else:
                    label = key
                sample_data[label] = sample[key][:]
        return name, sample_data

## Read and plot the data

Read the sample name and data

In [ ]:
# Define the file name (change this to your own file if needed)
fname = "tests/scan-0100_multi_demo.h5"

## Read the sample data
name, sample_data = get_sample_data(fname)

Plot the sample data (temperature vs time)

In [ ]:
plt.figure()
plt.title(name)
time = sample_data.pop('time (s)', None)
for label, data in sample_data.items():
    if time is not None and len(time) == len(data):
        plt.plot(time,data, label=label)
        plt.xlabel('time (s)')
    else:
        plt.plot(data, label=label)
plt.legend()
plt.show()

Read the 1D data

In [ ]:
## Read the azimuthally integrated 1D data
x, I, (x_label, I_label) = get_azint1d_data(fname)

Plot the few of the 1D diffraction patterns with an offset along the y-axis

In [ ]:
plt.figure()
plt.title(name)
offset = 0
for i in np.linspace(0,I.shape[0]-1,min(6,I.shape[0])).astype(int):
    plt.plot(x,I[i,:]+offset,label=f"frame {i}")
    offset += np.max(I)*0.1
plt.legend()
plt.xlabel(x_label)
plt.show()

Read the 2D data

In [ ]:
## Read the azimuthally binned 2D data
azi, x, I, (azi_label, x_label, I_label) = get_azint2d_data(fname)

Plot the average binned diffraction data ("cake") as a heatmap

In [ ]:
plt.figure()
plt.title(name)
plt.pcolormesh(x,azi,np.mean(I,axis=0),shading='auto')
plt.xlabel(x_label)
plt.ylabel(azi_label)
plt.title(I_label)
plt.colorbar(label=I_label)
plt.show()